In [147]:
import sigpy.plot as pl
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import scipy.io
from torch.autograd import Variable
% matplotlib notebook

In [2]:
import UFNet

In [6]:
mrf_dict = scipy.io.loadmat('/mikRAID/jtamir/projects/MRF_direct_contrast_synthesis/data/DictionaryAndSequenceInfo/fp_dictionary.mat')
# print(MRF_dic.keys())
fp_dict = mrf_dict['fp_dict']
t1_list = mrf_dict['t1_list']
t2_list = mrf_dict['t2_list']
N_dict = t1_list.shape[0]

In [7]:
def read_flipangles(flip_file):
    f = open(flip_file)
    flips =  [float(a) for a in f.readlines()]
    f.close()
    return np.array(flips)
flips = read_flipangles('/mikRAID/jtamir/projects/MRF_direct_contrast_synthesis/data/DictionaryAndSequenceInfo/flipangles.txt')
N_flip = len(flips)

In [8]:
fp_dic = np.hstack(list(fp_dict[0][0])).reshape((N_flip, 2, N_dict)).transpose((0, 2, 1))
# fp_dic = fp_dic[:,:,0] + 1j * fp_dic[:,:,1]
print(fp_dic.shape)

(500, 22031, 2)


In [9]:
fp_train = fp_dic.transpose((1,2,0))

In [10]:
fp_train.shape

(22031, 2, 500)

In [98]:
p = np.random.rand(22031,256)
p_norm = np.linalg.norm(p,axis=1)
p_normal = p/p_norm[None,:].T
B_tensor = torch.tensor(p_normal)

In [85]:
inp = torch.mm(B_tensor,t.double().transpose(0,1)).transpose(0,1)

In [87]:
a(inp,torch.LongTensor([3]))

tensor(10.0178, dtype=torch.float64, grad_fn=<NllLossBackward>)

In [78]:
a

CrossEntropyLoss()

In [ ]:
Memory_Bank

In [11]:
Net = UFNet.PixelNet(2)

In [103]:
B_tensor[100,:] = t

In [12]:
Net.train()

PixelNet(
  (conv1): Conv1d(2, 16, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=3840, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
)

In [105]:
fp_train.shape

(22031, 2, 500)

In [13]:
b = torch.tensor(fp_train[1000,:,:])

In [14]:
p = b.unsqueeze(0)

In [16]:
t = Net(p)

In [102]:
t.shape

torch.Size([1, 256])

In [97]:
print("Starting Epoch: %d" %2)

Starting Epoch: 2


In [104]:
p = np.random.rand(22031,256)
p_norm = np.linalg.norm(p,axis=1)
p_normal = p/p_norm[None,:].T
B_tensor = torch.tensor(p_normal)

In [185]:
B_tensor_cuda.detach()

tensor([[-0.0039,  0.9980, -0.0039,  ..., -0.0039, -0.0039, -0.0039],
        [ 0.0692,  0.9589, -0.0104,  ..., -0.0175, -0.0276, -0.0300],
        [ 0.1031,  0.9717, -0.0049,  ..., -0.0131, -0.0215, -0.0275],
        ...,
        [ 0.0910,  0.0860,  0.0129,  ...,  0.0092,  0.0177,  0.0103],
        [ 0.0703,  0.0598,  0.0907,  ...,  0.0319,  0.0062,  0.0959],
        [ 0.0636,  0.0984,  0.0485,  ...,  0.0610,  0.0659,  0.0979]],
       device='cuda:0', dtype=torch.float64)

In [233]:
# Here we train the network
n_dictionary = 22031
n_dimension = 256
tau = 0.07
def train_net(net,
              epochs=5,
              batch_size=1,
              lr=0.001):
    optimizer = torch.optim.SGD(net.parameters(), lr,
                                momentum=0.9,
                                weight_decay=1e-4)
    criterion = torch.nn.CrossEntropyLoss()
    loss_all = list([])
    for epoch in range(epochs):
        net.train()
        epoch_loss = 0
        print("Starting Epoch: %d" %(epoch+1))
        for index in range(n_dictionary):
            input_dic = fp_train_cuda[index,:,:].unsqueeze(0)
            output_dic = net(input_dic)
            
            
            ## error: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.
#             print(output_dic)
#             B_tensor_cuda.requires_grad = False
            output_dic1 = torch.mm(output_dic,B_tensor_cuda.detach().float().transpose(0,1))/tau
#             print(output_dic.requires_grad)
#             print(output_dic.shape)
            loss = criterion(output_dic1,torch.LongTensor([index]).cuda())
            if index % 1000 == 0:
                print(loss.item())
                loss_all.append(loss.item())
            epoch_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            B_tensor_cuda[index,:] = output_dic
        print('Epoch finished ! Loss: {}'.format(epoch_loss / 22031))
        torch.save(net.state_dict(),
                           '/home/kewang/checkpoints_unf/' + 'CP{}.pth'.format(epoch + 1))
        np.save("loss_all.npy",np.array(loss_all))
        print('Checkpoint {} saved !'.format(epoch + 1))

In [209]:
net_uf = UFNet.PixelNet(2)

In [210]:
net_ufcuda = net_uf.cuda()

In [211]:
fp_train_cuda = torch.tensor(fp_train).cuda()

In [212]:
B_tensor_cuda = B_tensor.cuda()

In [ ]:
train_net(net_ufcuda,1000,1,0.03)

Starting Epoch: 1
23.67812728881836
7.0704145431518555
7.534411430358887
7.770453453063965
8.029420852661133
8.102659225463867
7.91826057434082
7.784417152404785
7.009523391723633
5.8642988204956055
7.856454849243164
8.137837409973145
8.168560981750488
8.165803909301758
8.142561912536621
8.191444396972656
8.191556930541992
6.424860000610352
5.887223243713379
5.671389579772949
8.18789005279541
8.759794235229492
9.13525390625
Epoch finished ! Loss: 7.495940938484414
Checkpoint 1 saved !
Starting Epoch: 2
23.37740707397461
5.922143936157227
6.813292503356934
7.033307075500488
7.223241806030273
7.222947120666504
7.114246368408203
6.94333553314209
5.98677921295166
6.057112693786621
7.219635963439941
7.504720687866211
